In [16]:
import xarray
import hvplot.xarray
from dask.distributed import Client
import numpy as np
import pandas as pd
import datetime
import panel as pn
import holoviews as hv
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool
pn.extension()

In [17]:
#infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_Out_Oct17_Dec3_2018_67391492\summary\hourly_summary.nc'
#infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_In_Dec3_2018_Jan30_2019_67391491\summary\hourly_summary.nc'
#infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_In_Jan_April_2019_67674121\summary\hourly_summary.nc'
infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_in_April_July2019_1342218252\summary\hourly_summary.nc'
#infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_In_July_October2019_67674121\summary\hourly_summary.nc'

#infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_in_April_July2019_1342218252\summary\hourly_summary.nc'

global ds
ds = xarray.open_dataset(infile)


In [18]:
#indir = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_In_summaries'
#indir = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\RCA_Out_Oct17_Dec3_2018_67391492\summary'
#global ds
#ds = xarray.open_mfdataset(indir + "/*.nc",
#                          coords=['date'],
#                          combine='nested',
#                          compat='override',
#                          )
#ds

In [19]:
# widgets
class_list = list(ds.data_vars)
class_name  = pn.widgets.Select(name='Class',
                                value=class_list[0],
                                options=class_list
                               )
threshold = pn.widgets.DiscreteSlider(name='Confidence', options=list(ds.confidence.data), value=list(ds.confidence.data)[0])

# Stream
# Declare Tap stream with heatmap as source and initial values
#posxy = hv.streams.Tap(source=heatmap, x=1951, y='New York')
tap = hv.streams.Tap(x=0,y=0)

file_input = pn.widgets.FileInput()
file_selector = pn.widgets.FileSelector('~')

In [20]:
@pn.depends(class_name,threshold)
def plot1D(class_name,threshold,time_resolution='H'):
    global ds
    if time_resolution == 'H':
        plot = ds[class_name].sel(confidence=threshold,method='nearest').hvplot()
    elif time_resolution == 'D':
        plot = ds[class_name].sel(confidence=threshold,method='nearest').resample(date='D').sum().hvplot(kind='step', width=820)
    else:
        print('Time resolution not supported. Use "H" for hours, or "D" for days.')
    return plot

In [21]:
@pn.depends(class_name,threshold)
def plot2D(class_name,threshold):
    global ds
    ds1 = ds[class_name].sel(confidence=threshold,method='nearest')
    # Date vector
    date = list(ds1.resample(date='D').groups.keys())
    # hour of day vector
    #hour = pd.date_range("00:00", freq="1H", periods=24).time
    hour = range(0,24,1)

    # detections matrix
    detections = np.empty((len(hour),len(date),))
    detections[:] = np.nan
    detec_matrix = xarray.DataArray(detections, coords={'hour': hour, 'date': date}, dims=['hour','date'])

    # Fill in 2D Array
    for x in ds1:
        fulldate = pd.to_datetime(x.date.data)
        day_str = datetime.datetime.strftime(fulldate,'%Y-%m-%d')
        hour = fulldate.time().hour
        nDetec = x.data
        detec_matrix.loc[hour,day_str] = nDetec

    # Sum of detec per day
    detec_sum = detec_matrix.sum(dim='hour')
    
    #formatter = DatetimeTickFormatter(months='%B %Y')
    #formatter = DatetimeTickFormatter(months='%B')
    
    image_plot = detec_matrix.hvplot.image(cmap='rainbow',
                                           width=800,
                                           xlabel='Date',
                                           ylabel='Hour of the day',
                                           clabel='Number of detections',
                                           tools =['tap'],
                                           responsive=False,
                                           #xformatter=formatter,
                                          )
    
    #hover = image_plot.select(dict(type=HoverTool))
    #hover.tooltips = [("Date", "@x"),("Hour", "@y"),]
    #image_plot.xformatter=formatter
    tap.source = image_plot
    
    return image_plot
    #return detec_matrix.hvplot.heatmap(x='Date', y='Hour of day',cmap='rainbow',width=800)    

In [22]:
widgets = pn.Column(file_input, class_name,threshold)
plots = pn.Column(plot2D,plot1D)
app = pn.Row(widgets,plots)
app.show()
#widgets

Launching server at http://localhost:60096


In [ ]:
tap

In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', width=90)

In [ ]:
df = pd.read_csv('http://assets.holoviews.org/data/diseases.csv.gz', compression='gzip')
dataset = hv.Dataset(df, vdims=('measles','Measles Incidence'))

# Declare HeatMap
heatmap = hv.HeatMap(dataset.aggregate(['Year', 'State'], np.mean),
                     label='Measles Incidence').select(Year=(1928, 2002))

# Declare Tap stream with heatmap as source and initial values
posxy = hv.streams.Tap(source=heatmap, x=1951, y='New York')

In [ ]:
# Define function to compute histogram based on tap location
def tap_histogram(x, y):
    return hv.Curve(dataset.select(State=y, Year=int(x)), kdims='Week',
                   label='Year: %s, State: %s' % (x, y))

tap_dmap = hv.DynamicMap(tap_histogram, streams=[posxy])

(heatmap + tap_dmap).opts(
    opts.Curve(framewise=True, height=500, line_color='black', width=375, yaxis='right'),
    opts.HeatMap(cmap='RdBu_r', fontsize={'xticks': '6pt'}, height=500,
                 logz=True, tools=['hover'], width=700, xrotation=90)
)

In [ ]:
dataset

In [ ]:
df

In [ ]:
formatter

In [ ]:
formatter = DatetimeTickFormatter(days ='%m')
formatter.days

In [ ]:
dd=plot2D('FS',0.5)

In [ ]:
hover = dd.select(dict(type=HoverTool))
hover.tooltips = [("Date", "@x{int}"),("Hour", "$y"),]

In [ ]:
dd